Let's download data as a first step to win time 😀

In [ ]:
import kagglehub
CRC_FOLDER = imrankhan77_crc_val_he_7k_path = kagglehub.dataset_download('imrankhan77/crc-val-he-7k')
print(CRC_FOLDER)
NCT_FOLDER = imrankhan77_nct_crc_he_100k_path = kagglehub.dataset_download('imrankhan77/nct-crc-he-100k')
print(NCT_FOLDER)

print('Data source import complete.')

# Histology tissue classification using Deep Learning models

The main purpose of this practical notebook is to apply a deep learning model to solve a basic tissue classficiation task in histology images.

For this session, we will use two colon adenocarcinoma datasets:  [nct-crc-he-100k](https://www.kaggle.com/datasets/imrankhan77/nct-crc-he-100k) and [crc-val-he-7k](https://www.kaggle.com/datasets/imrankhan77/crc-val-he-7k). These two datasets contain tissue tiles classified with nine labels:

*   ADI: Adipose
*   STR: Strome
*   TUM: Tumour
*   NORM: Normal tissue
*   BACK: Background
*   LYM: Lymphome
*   DEB: Debris
*   MUC: Mucose
*   MUS: Musculus

Each dataset has 100K and 7K tiles images and are mutually excluyent between them. i.e, There aren't any slide repeated between NCT and CRC




In addition, we will use a foundational model for histopathology as feature extractor: UNI *(Chen, 2024)*. You can check for more information about it on [Huggingface](https://huggingface.co/MahmoodLab/UNI) 🤗 and the [arcticle](https://www.nature.com/articles/s41591-024-02857-3)


We will apply some Machine Learning algorithms using the extracted features, and then, select and optimize for our final prediction

LET'S GO

## Dependences, tokens, data and GPUs



First of all, let's install [Optuna](https://optuna.org/). We will use it later for model optimization. All other packages and libraries are included in google collab base environment, so we don't need to install it.

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00


We will disable warnings for to simplify the visualization

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Google collab provides free but limited access to a NVIDIA T4 (suficient for this demo). For an unlimited use of NVIDIA P40 on cloud, you can this Kaggle [notebook](https://www.kaggle.com/code/gabrielcabas/histology-tissue-classification).  Kaggle provides two NVIDIA T4 and one  NVIDIA P100 . Otherwise, you can always download this notebook and use it on your laptop or adapt it to use on an HPC.

Our last dependence is the permission of the authors of UNI to use this model. **Only for this time** we will use my token as a group. If you decide to use it further please request access via Huggingface.
Use Secrets to insert and import the token

In [ ]:
#Token
from huggingface_hub import login
from google.colab import userdata
token = userdata.get('huggingface_token')
login(token = token)

And import the necessary libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os #Operative system calls and files
import matplotlib.pyplot as plt #Data visualization
import seaborn as sns #Wrapper for plt
from PIL import Image #Image open
import tqdm

In [ ]:
#ML dependences
import torch
import torchvision
from torch.utils.data import DataLoader
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from torchvision import transforms
from numba import cuda
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import optuna

This is just a function for to clear the GPU and free memory.

In [ ]:
def clear_gpu(): #Function to clear cache in emergency case D:
    torch.cuda.empty_cache()

In [ ]:
clear_gpu()

And because everyone loves gg-plot, we will import this styles

In [ ]:
plt.style.use('ggplot')

## Exploratory data analysis

In [ ]:
NCT_FOLDER = f"{NCT_FOLDER}/NCT-CRC-HE-100K"
CRC_FOLDER = f"{CRC_FOLDER}/CRC-VAL-HE-7K"

 This function only list files in main folders and stores it on a dataframe

In [ ]:
def list_files(dataset_folder):
    data = []
    for folder, subdir, file in os.walk(dataset_folder):
        label = os.path.basename(folder)
        if file != []:
            tiles = pd.DataFrame(file, columns=["tile"])
            tiles["label"] = label
            data.append(tiles)
    data = pd.concat(data)
    return data

We will use NCT dataset for training

In [ ]:
nct_df = list_files(NCT_FOLDER)
nct_df

 And CRC for testing

In [ ]:
crc_df = list_files(CRC_FOLDER)
crc_df

Let's show the labels distribution over tiles

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.title("NCT-CRC-HE-100k")
sns.histplot(nct_df, hue="label", x="label")
plt.subplot(1,2,2)
plt.title("CRC-VAL-HE-100k")
sns.histplot(crc_df, hue="label", x="label")

This function show some random tiles per label in each dataset

In [ ]:
def show_tiles(df, dataset_folder):
  plt.figure(figsize= (15, 30))
  sample_n = 5
  plt.suptitle(dataset_folder.split("/")[-1], size=20)
  labels = df.label.unique()
  for id_label, label in enumerate(labels):
      sample = df[df.label == label].sample(5).reset_index(drop=True)
      for index, row in sample.iterrows():
          path = f"{dataset_folder}/{label}/{row.tile}"
          img = Image.open(path)
          plt.subplot(len(labels), sample_n, (id_label*sample_n) + index+1)
          if index == 0:
              plt.ylabel(label)
          plt.imshow(img)
  plt.tight_layout()

In [ ]:
show_tiles(nct_df, NCT_FOLDER)

In [ ]:
show_tiles(crc_df, CRC_FOLDER)

## Import model

Let's use cuda for to import model and data in the GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

And to use timm (Huggingface) to download UNI model and mount it in the GPU

In [ ]:
feature_extractor = timm.create_model("hf-hub:MahmoodLab/uni", pretrained=True, init_values=1e-5, dynamic_img_size=True)
transform = create_transform(**resolve_data_config(feature_extractor.pretrained_cfg, model=feature_extractor))
feature_extractor.to(device)

We will use standard image normalization

In [ ]:
transform = transforms.Compose(
        [
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    )

Let's use pytorch to import Imagefolders and create generators to evaluate the model.

In [ ]:
image_folder = torchvision.datasets.ImageFolder(NCT_FOLDER, transform = transform)
train_loader = DataLoader(image_folder, batch_size=256, shuffle=True)

In [ ]:
image_folder = torchvision.datasets.ImageFolder(CRC_FOLDER, transform = transform)
test_loader = DataLoader(image_folder, batch_size=256, shuffle=True)

## Feature extraction

The labels and the ordinal encoding

In [ ]:
image_folder.class_to_idx
idx_to_class = {v: k for k, v in image_folder.class_to_idx.items()}
idx_to_class

This functions simply applies the model to the batch of images, we will limit this process to a low number of batches.
This is the time consuming process of the notebook.
To use it in all the images should take ~ 1:30 hrs.

In [ ]:
def feature_extraction(loader, sample_length):
    with torch.inference_mode():
        embeddings = []
        targets = []
        it = iter(loader)
        for index in tqdm.tqdm(range(sample_length)):
            a = next(it)
            x = a[0].to(device)
            targets.append(a[1])
            features = feature_extractor(x).cpu().detach().numpy()
            embeddings.append(features)
        embeddings = np.concatenate(embeddings)
        targets = np.concatenate(targets)
        return embeddings, targets

Let's use the function and extract the features on X_train and X_test

In [ ]:
X_train, y_train = feature_extraction(train_loader, 20)
X_test, y_test = feature_extraction(test_loader, 10)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Features visualization

We will use a PCA to show how does the features distribute and if there's a trivial separation (clusters) between them

In [ ]:
pca = PCA(n_components = 2)
transformed = pca.fit_transform(X_train)
trans = pd.DataFrame(transformed)
trans["target"] = y_train
trans.target = trans.target.replace(idx_to_class)
trans

As you might see, the explained variance for the first two components is slow. Neverhteless there are two highly separated clusters in the III and IV quadrant. These are LYM and BACK. You can watch the sample images to conclude if they are very different images from rest.

The other separation is not trivial, so we need to apply a more advanced classification. Here is where Machine Learning comes to action

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.scatterplot(data = trans, x= 0, y= 1 , hue= "target", alpha=1, s=20)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.title("PCA features")
plt.xlabel(f"PC1: {round(pca.explained_variance_ratio_[0] * 100, 2)}")
plt.ylabel(f"PC2: {round(pca.explained_variance_ratio_[1] * 100, 2)}")
plt.tight_layout()

## Train model
We can use Scikit-learn to explore some algorithms, and to decide how is going to be our optimized model.


In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def train_model(model, X_train, y_train, X_test, y_test):
  model.fit(X_train, y_train)
  train_acc = model.score(X_train, y_train)
  cv_acc = cross_val_score(model, X_train, y_train) #We wil apply k-fold cross validation on X_train to obtain a most consistent metric.
  test_acc = model.score(X_test, y_test)
  print("cv_accuracy", cv_acc.mean(), sep=":")
  print("test_accuracy", test_acc, sep=":")

In [ ]:
train_model(RandomForestClassifier(), X_train, y_train, X_test, y_test)

In [ ]:
train_model(AdaBoostClassifier(), X_train, y_train, X_test, y_test)

In [ ]:
train_model(MLPClassifier(), X_train, y_train, X_test, y_test)

## Model optimization

Finally, we will find a better hyperparameters combination using Optuna. This is a bayesian optimizer that searches over some given options and tries to reach the best performance

We will construct a multi layer perceptron using only three hidden layers. This layers may have a different number of neurons and the number of different combinations is huge.

In [ ]:
def objective(trial):
    neurons_1 = trial.suggest_int("neurons_1", 50, 200)
    neurons_2 = trial.suggest_int("neurons_2", 50, 200)
    neurons_3 = trial.suggest_int("neurons_3", 50, 200)
    classifier_obj = MLPClassifier(hidden_layer_sizes=(neurons_1, neurons_2, neurons_3), early_stopping=True, learning_rate="adaptive")
    classifier_obj.fit(X_train, y_train)
    accuracy = classifier_obj.score(X_test, y_test)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

Once the optimization is done, we can get the best combination of hyperparameters and the metric obtained.

In [ ]:
study.best_params

This is a very tinny example to apply Deep learning in histology, so it's not surprising that the metrics on this datasets are usually very high.

In [ ]:
round(study.best_value, 3)

Let's retrain our model with our optimizer hyperparameters

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(study.best_params["neurons_1"], study.best_params["neurons_2"], study.best_params["neurons_3"]),
                      early_stopping=True, learning_rate="adaptive")
model.fit(X_train, y_train)
model

We can visualize the search over time

In [ ]:
optuna.visualization.plot_optimization_history(study)

Always is a good practice to watch the confussion matrix, so we can analyze the performance and the possible errors our model is doing.

In [ ]:
cf_matrix = confusion_matrix(y_test, model.predict(X_test))
sns.heatmap(cf_matrix, annot=True, cmap='Blues',fmt='')
plt.title("Test Confussion matrix")
plt.ylabel("Y true")
plt.xlabel("Y pred")
plt.yticks(np.arange(len(idx_to_class))+0.5, list(idx_to_class.values()))
plt.xticks(np.arange(len(idx_to_class))+0.5, list(idx_to_class.values()))

And that's all. Good luck!